In [1]:
import random
from datetime import datetime, date, timedelta
from itertools import chain
import pandas as pd
import networkx as nx
from itertools import combinations  
import requests 
import numpy as np

NUM_DOCS = 10
NUM_PATIENTS = 100
MAX_NOTES_PER_DIS = 10
MAX_STAY_PER_PATIENT = 6

def get_random_names(count):
    response = requests.get(f'https://randomuser.me/api?results={count}&inc=name&nat=us,gb,es')
    names = [f"{user['name']['first']} {user['name']['last']}" for user in response.json()['results']]
    return names

drs = [name for name in get_random_names(NUM_DOCS)]

patients = [name for name in get_random_names(NUM_PATIENTS)]

def get_date():
    month = random.randint(1,12)
    day = random.randint(1,28)
    return datetime(2019, month, day)

def generate_notes_for_discharge(discharge):
    notes = [(discharge[0], random.choice(drs), discharge[2]) 
             for _ in range(0, random.randint(1,MAX_NOTES_PER_DIS))]
    return notes

def get_date_range(start, end):
    return np.arange(start, end, np.timedelta64(1, 'D'))

def generate_notes(discharges):
    for d in discharges:
        yield generate_notes_for_discharge(d)
        
def get_random_timedelta():
    return timedelta(days=random.randint(1,MAX_STAY_PER_PATIENT))

discharges = [(i, p, d := get_date(), d + get_random_timedelta())
    for i,p in enumerate(patients)]

notes = list(chain.from_iterable(generate_notes(discharges)))

dis_df = pd.DataFrame(discharges, columns=['id', 'patient', 'arrive_date', 'discharge_date'])

note_df = pd.DataFrame(notes, columns=['discharge_id', 'dr', 'date'])

dis_df.to_csv('./data/discharges.csv', index=False, sep=',')

note_df.to_csv('./data/notes.csv', index=False, sep=',')
